In [2]:
import numpy as np
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt

class EquilState:
    def __init__(self, components, mole_fraction):
        self.components = list(components.keys())
        self.mole_fraction = np.array(mole_fraction)
        self.n = len(self.components)
        self.R = 8.314
        
        self.Tc = np.array([components[comp]['Tc'] for comp in self.components])
        self.Pc = np.array([components[comp]['Pc'] for comp in self.components])
        self.omega = np.array([components[comp]['omega'] for comp in self.components])
        self.Mw = np.array([components[comp]['Mw'] for comp in self.components])
        
        self.kij = np.zeros((self.n, self.n))
        np.fill_diagonal(self.kij, 0)
        
        self.ai = np.zeros(self.n)
        self.bi = np.zeros(self.n)
        self.alpha_i = np.zeros(self.n)

    def Coeff_PR(self, T):
        self.ai = 0.45724 * (self.R**2 * self.Tc**2) / self.Pc
        self.bi = 0.07780 * self.R * self.Tc / self.Pc

        mi = np.zeros(self.n)
        for i in range(self.n):
            if self.omega[i] <= 0.49:
                mi[i] = 0.37464 + 1.54226 * self.omega[i] - 0.26992 * self.omega[i]**2
            else:
                mi[i] = 0.3796 + 1.485 * self.omega[i] - 0.1644 * self.omega[i]**2 + 0.01667 * self.omega[i]**3
        
        self.alpha_i = (1 + mi * (1 - np.sqrt(T / self.Tc)))**2

    def MixParam(self, z):
        a_mix = 0
        for i in range(self.n):
            for j in range(self.n):
                a_mix += z[i] * z[j] * np.sqrt(self.ai[i] * self.ai[j]) * (1 - self.kij[i,j])
        b_mix = np.sum(z * self.bi)
        return a_mix, b_mix

    def Solve_PR(self, a, b, P, T):
        coeffs = [
            P,
            -(self.R * T - P * b),
            (a - 2 * self.R * T * b - 3 * P * b**2),
            -(a*b - self.R*T*b**2 - P*b**3)
        ]
        
        roots = np.roots(coeffs)
        real_roots = np.real(roots[np.isreal(roots)])
        pos_roots = real_roots[real_roots > 0]
        
        return pos_roots

    def Volatility(self, x, phase, P, T):
        
        a_mix, b_mix = self.MixParam(x)
        roots = self.Solve_PR(a_mix, b_mix, P, T)
        
        if phase == 'liquid':
            valid_roots = roots[roots > b_mix * 1.001]
            if len(valid_roots) == 0:
                return np.ones(self.n) * np.nan, np.nan, np.nan, np.nan, np.nan
            V = np.min(valid_roots)
        else:
            if len(roots) == 0:
                return np.ones(self.n)*np.nan, np.nan, np.nan, np.nan, np.nan
            V = np.max(roots)
        
        Z = P * V / (self.R * T)
        AA = a_mix * P / (self.R * T)**2
        BB = b_mix * P / (self.R * T)
        
        if (Z - BB) <= 0 or (Z + (1 + np.sqrt(2)) * BB) <= 0 or (Z + (1 - np.sqrt(2)) * BB) <= 0:
            return np.ones(self.n)*np.nan, np.nan, np.nan, np.nan, np.nan
        
        phi = np.zeros(self.n)
        for i in range(self.n):
            summ = 0
            for j in range(self.n):
                summ += x[j] * np.sqrt(self.ai[i] * self.ai[j]) * (1 - self.kij[i,j])
            
            t1 = self.bi[i] / b_mix * (Z - 1)
            t2 = -np.log(Z - BB + 1e-12)
            t3 = -AA / (2 * np.sqrt(2) * BB + 1e-12) * (2 * summ / a_mix - self.bi[i] / b_mix)
            t3 *= np.log((Z + (1 + np.sqrt(2)) * BB + 1e-12) / (Z + (1 - np.sqrt(2)) * BB + 1e-12))
            
            phi[i] = np.exp(t1 + t2 + t3)
        
        rho = np.sum(x * self.Mw) * 1e-3 / V
        
        return phi, Z, V, rho

    def Stability(self, z, P, T, phase):
        phi_z, _, _, _, _ = self.Volatility(z, phase, P, T)
        
        if phase == 'liquid':
            y_test = z * np.exp(np.random.normal(0, 0.1, size=len(z)))
            y_test = y_test / np.sum(y_test)
            phi_y, _, _, _, _ = self.Volatility(y_test, 'vapor', P, T)
            R = np.sum(y_test * (np.log(y_test) + np.log(phi_y) - np.log(z) - np.log(phi_z)))
        else:
            x_test = z * np.exp(np.random.normal(0, 0.1, size=len(z)))
            x_test = x_test / np.sum(x_test)
            phi_x, _, _, _, _ = self.Volatility(x_test, 'liquid', P, T)
            R = np.sum(x_test * (np.log(x_test) + np.log(phi_x) - np.log(z) - np.log(phi_z)))
        
        return R < -1e-3

    def Equil(self, P, T, max_iter = 100, tol = 1e-12):
        self.Coeff_PR(T)
        
        K = np.exp(np.log(self.Pc / P) + 5.373 * (1 + self.omega) * (1 - self.Tc / T))
        x = np.clip(self.mole_fraction / (1 + (K - 1)), 1e-12, 1)
        y = K * x
        
        x = x / np.sum(x)
        y = y / np.sum(y)
        
        for iter in range(max_iter):
            phi_liq, Z_liq, V_liq, rho_liq = self.Volatility(x, 'liquid', P, T)
            phi_vap, Z_vap, V_vap, rho_vap = self.Volatility(y, 'vapor', P, T)
            
            if np.any(np.isnan(phi_liq)) or np.any(np.isnan(phi_vap)):
                K = np.exp(np.log(self.Pc / P) + 5.373*(1 + self.omega)*(1 - self.Tc / T))
                y = K * x
                continue
                
            K_new = phi_liq / phi_vap
            error = np.max(np.abs(np.log(K_new / K)))
            
            if error < tol:
                beta = np.clip(np.mean((self.mole_fraction - x)/(y - x + 1e-12)), 0, 1)
                return self.Result(x, y, beta, 'ok', P, T, iter + 1, error)
            
            K = K_new
            y = K * x
            y = y / np.sum(y)
        
        return self.Result(None, None, None, 'failed - max iterations', P, T)

    def Result(self, x, y, beta, status, P, T, iter = None, error = None):
        result = {
            'status': status,
            'phase_fraction': float(beta) if beta is not None else None
        }
        
        if x is not None:
            phi, Z, V, rho = self.Volatility(x, 'liquid', P, T)
            result['liquid'] = {
                'mole_fraction': {name: float(val) for name, val in zip(self.components, x)},
                'Z': float(Z),
                'density': float(rho)
            }
        
        if y is not None:
            phi, Z, V, rho = self.Volatility(y, 'vapor', P, T)
            result['vapor'] = {
                'mole_fraction': {name: float(val) for name, val in zip(self.components, y)},
                'Z': float(Z),
                'density': float(rho)
            }
        
        if iter is not None:
            result['iterations'] = int(iter)
        if error is not None:
            result['error'] = float(error)
            
        return result

if __name__ == "__main__":
    components = {
        'Methane': {'Tc': 190.6, 'Pc': 45.99e5, 'omega': 0.011, 'Mw': 16.04},
        'Propane': {'Tc': 369.8, 'Pc': 42.48e5, 'omega': 0.152, 'Mw': 44.10},
        'Hexane': {'Tc': 507.6, 'Pc': 30.25e5, 'omega': 0.301, 'Mw': 86.18}
    }
    
    mole_fraction = [0.2, 0.3, 0.5]
    
    system = EquilState(components, mole_fraction)
    result = system.Equil(1.01325e5, 288.15)
    
    print("Результаты расчета фазового равновесия:")
    if result['status'] == 'ok':
        print("\nЖидкая фаза:")
        print(f"Состав: {result['liquid']['mole_fraction']}")
        print(f"Z-фактор: {result['liquid']['Z']:.4f}")
        print(f"Плотность: {result['liquid']['density']:.4f} кг/м3")
        
        print("\nПаровая фаза:")
        print(f"Состав: {result['vapor']['mole_fraction']}")
        print(f"Z-фактор: {result['vapor']['Z']:.4f}")
        print(f"Плотность: {result['vapor']['density']:.4f} кг/м3")
        
        print(f"\nДоля паровой фазы: {result['phase_fraction']:.4f}")
    else:
        print(f"Ошибка расчета: {result['status']}")

Результаты расчета фазового равновесия:

Жидкая фаза:
Состав: {'Methane': 0.0006722650902167949, 'Propane': 0.03967177419345307, 'Hexane': 0.9596559607163301}
Z-фактор: 0.0060
Плотность: 592.9708 кг/м3

Паровая фаза:
Состав: {'Methane': 0.028186770454680904, 'Propane': 0.21640986544788549, 'Hexane': 0.7554033640974336}
Z-фактор: 0.9649
Плотность: 3.2918 кг/м3

Доля паровой фазы: 1.0000
